In [1]:
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import os
import re
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import glob
import pandas as pds
N = 100
# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
reff = np.zeros((N,1))
from keras import metrics
import math
sus =  np.zeros((N,1))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt # this is used for the plot the graph

In [2]:
df2 = pd.DataFrame(columns=[ 'id','status'])


# Limpiando el corpus - tardó 74 minutos en limpiarse el corpus

Esto solo debe correrse una vez, para no correrlo cada vez que abramos el archivo y esperar 74 mins cada vez,  decidimos guardar el resultado de un corpus limpio en el archivo "corpus_processed.txt".

In [3]:

# Stopwords
nltk.download("stopwords") 
from nltk.corpus import stopwords 

# # Stemming --> get root of the word
# from nltk.stem.porter import PorterStemmer 
# from nltk.stem import SnowballStemmer # best stemmer


# # Lemmatization --> get context
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# from nltk.stem.wordnet import WordNetLemmatizer

# #eliminarAcentos = lambda s: s.decode('utf-8').translate(str.maketrans("áàäéèëíìïòóöùúüÀÁÄÈÉËÌÍÏÒÓÖÙÚÜ", "aaaeeeiiiooouuuAAAEEEIIIOOOUUU")).encode('utf-8')


# corpus = [] 

# for i in list(df.loc[:,"text"]):
#     # 0. Quitamos acentos:
#     #text = eliminarAcentos(i)
#     # 1. Quitamos todo lo que no sea una letra:
#     text = re.sub("[^a-zA-Z]", " ", i.decode("utf-8"))
#     # 2. Todo en minúsculas:
#     text = text.lower()
#     # 2.1 Quitamos espacios multiples y dejamos solo uno:
#     text = re.sub("[\s]+", " ", text)

#     # 2.2 Extra: Quitar palabras de menos de 3 letras y espacios al principio y final
#     text = re.sub(r"\W*\b\w{1,3}\b", "", text)
#     text = text.strip()

#     # 3. Separamos oraciones para obtener puras palabras
#     text = text.split()

#     # 4. Extraer el significado detrás de cada palabra Stemming o Lemmatization y eliminar ruido
#     #stemmer = SnowballStemmer('spanish') # Best stemmer
#     #ps = PorterStemmer()
#     lemmatizer = WordNetLemmatizer()

#     # 4.1 Además queremos quitar palabras irrelevantes como artículos
#     all_stopwords = stopwords.words() 
#     #all_stopwords.remove("palabra") # --> para excepciones que no queramos incluir

#     # 4.2 Unimos el Stemmer/Lematization para cada palabra que no esté dentro de la lista de stopwords
#     #text = [stemmer.stem(word) for word in text if word not in set(all_stopwords)]
#     text = [lemmatizer.lemmatize(word) for word in text if word not in set(all_stopwords)]
#     #text = [word for word in text if word not in set(all_stopwords)]

  
#     # 5. Unir oraciones con espacios entre cada palabra
#     text = " ".join(text)
#     corpus.append(text)

# corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matts\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#guardando el código en un archivo de texto
# filename = "corpus_processed.txt"  

# with open(filename, "w") as file:
#     for item in corpus:
#         file.write(str(item) + "\n")

In [5]:
#importando el corpus procesado 
df = pd.read_table("corpus_processed.txt", header=None, names=["text"])


# Importar datos

Importando datos de fuente original del articulo y machine generated. Se esta omitiendo el importar el texto de los articulos porque ya tenemos el corpus procesado en "df" pero es importante mencionar que inicialmente se hizo el mismo para la columna "text"

In [6]:
#para importar datos de machine generated solo hace falta cambiar "mg" por "og" y
folder_path = "wikipedia_documents_test/machined/og/"

# Encontrar folder
files = glob.glob(os.path.join(folder_path, "*.txt"))
# Iterate over each txt file
for file in files:
  
    # Open the PDF file
    
    filename = os.path.basename(file)
    filename = filename.split('-')[0]
    
    txt = open(file, 'rb')

    # Leer contenido de cada txt
    content = txt.read()

    txt.close()
    
    #se agrego incialmente content a la columna text pero se esta omitiendo ese paso ahora
    df2.loc[len(df2)] = [ filename, "og"]
    
df2

,id,status
0,10021497,og
1,10025,og
2,100390,og
3,1004070,og
4,1004429,og
...,...,...
1985,99228,og
1986,9923141,og
1987,994456,og
1988,9974682,og


In [7]:
#para importar datos de machine generated solo hace falta cambiar "mg" por "og" y
folder_path = "wikipedia_documents_test/machined/mg/"

# Encontrar folder
files = glob.glob(os.path.join(folder_path, "*.txt"))
# Iterate over each txt file
for file in files:
  
    # Open the PDF file
    
    filename = os.path.basename(file)
    filename = filename.split('-')[0]
    
    txt = open(file, 'rb')

    # Leer contenido de cada txt
    content = txt.read()

    txt.close()
    
    #se agrego incialmente content a la columna text pero se esta omitiendo ese paso ahora
    df2.loc[len(df2)] = [ filename, "mg"]
    
df2

,id,status
0,10021497,og
1,10025,og
2,100390,og
3,1004070,og
4,1004429,og
...,...,...
3975,99228,mg
3976,9923141,mg
3977,994456,mg
3978,9974682,mg


In [8]:
df = df.join(df2)

Observamos que el dataset esta bien balanceado

Hacemos categorical encoding para el target a predecir ya que la red neuronal lo requiere

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["status"] = le.fit_transform(df["status"])

In [10]:
df

,text,id,status
0,washington state route state route short state...,10021497,1
1,banner europe european flag flag europe offici...,10025,1
2,creation creation formally prelature holy cros...,100390,1
3,citadel town city osijek croatia safeguarded b...,1004070,1
4,liam miller liam william miller february febru...,1004429,1
...,...,...,...
3975,long range desert group long range desert grou...,99228,0
3976,vaughn jonathan stewart vaughn born march form...,9923141,0
3977,augusto bastos augusto bastos june april parag...,994456,0
3978,trondheim airport station trondheim airport st...,9974682,0


# mg = 1 , og = 0

Hacemos shuffle a los datos

In [11]:
df = df.sample(frac=1).reset_index(drop=True)
df

,text,id,status
0,clarinet clarinet musical instrument family be...,6433,0
1,franz burgmeier franz burgmeier born april lie...,11471512,0
2,chartjackers chartjackers british narrative ar...,25154219,1
3,violent wind tropical cyclone twister caused r...,36619573,1
4,leccinum rugosiceps leccinum rugosiceps common...,45459609,0
...,...,...,...
3975,returning home diddy dirty money melody return...,30010737,1
3976,euro note euro note esteem euro banknote utili...,24190481,1
3977,uncanny tale canadian pulp magazine uncanny ta...,15738365,0
3978,krista branch krista branch american vocalist ...,33562778,1


Vectorizamos palabras

In [12]:

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df[['text', 'id']].apply(lambda x: ' '.join(x), axis=1)).toarray()
y = df["status"].values

In [13]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
df

,text,id,status
0,clarinet clarinet musical instrument family be...,6433,0
1,franz burgmeier franz burgmeier born april lie...,11471512,0
2,chartjackers chartjackers british narrative ar...,25154219,1
3,violent wind tropical cyclone twister caused r...,36619573,1
4,leccinum rugosiceps leccinum rugosiceps common...,45459609,0
...,...,...,...
3975,returning home diddy dirty money melody return...,30010737,1
3976,euro note euro note esteem euro banknote utili...,24190481,1
3977,uncanny tale canadian pulp magazine uncanny ta...,15738365,0
3978,krista branch krista branch american vocalist ...,33562778,1


In [15]:
# Train y test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=29)

# Entrenamiento de la RNN

Una red neuronal artificial (ANN, por sus siglas en inglés) es un modelo matemático y computacional que se inspira en la estructura y el funcionamiento del cerebro humano para procesar información. Consiste en un conjunto de nodos interconectados, conocidos como neuronas artificiales, que reciben una entrada de datos, realizan un cálculo y producen una salida.

Cada neurona en una ANN tiene un conjunto de entradas ponderadas que se suman y se pasan a través de una función de activación no lineal para generar la salida de la neurona. La red está organizada en capas, donde cada capa se compone de múltiples neuronas. La primera capa se llama capa de entrada, la última capa se llama capa de salida, y las capas intermedias se llaman capas ocultas.

El proceso de entrenamiento de una ANN implica ajustar los pesos y los sesgos de las conexiones entre las neuronas para minimizar una función de pérdida. Esto se realiza utilizando técnicas de aprendizaje supervisado o no supervisado, en función del tipo de problema que se esté abordando.

En este caso usaremos las herramientas de Tensor Flow para poder crear nuestra ANN

-  'units' se refiere al número de neuronas en la capa densa que se está agregando a la red neuronal. Es decir, esta capa tendrá 200 neuronas en total. El número de unidades en una capa densa es un parámetro que se puede ajustar para controlar la complejidad y la capacidad de aprendizaje de la red neuronal.  units se refiere al número de neuronas en la capa densa que se está agregando a la red neuronal. Es decir, esta capa tendrá 200 neuronas en total. El número de unidades en una capa densa es un parámetro que se puede ajustar para controlar la complejidad y la capacidad de aprendizaje de la red neuronal.

- 'activation'  se refiere a la función de activación que se utilizará en la capa densa que se está agregando a la red neuronal.

    - La función de activación es una función matemática que se aplica a la salida de cada neurona en una capa para introducir no linealidad en la red neuronal. La función relu (unidad lineal rectificada) es una función de activación comúnmente utilizada que retorna 0 si la entrada es negativa y la entrada misma si es positiva o cero.

    - La función relu se usa a menudo porque puede ayudar a prevenir el problema de la desaparición del gradiente en el entrenamiento de redes neuronales profundas, lo que puede ocurrir cuando se usan funciones de activación que tienen gradientes muy pequeños en ciertas regiones.

    - La función Softmax es una función de activación comúnmente utilizada en la capa de salida de una red neuronal para producir una distribución de probabilidad sobre varias clases. La función Softmax es especialmente útil en problemas de clasificación multiclase, donde se desea predecir la probabilidad de pertenencia a varias clases diferentes. Por ejemplo, en una red neuronal para clasificar imágenes, la función Softmax puede utilizarse para producir una distribución de probabilidad sobre varias categorías de objetos (por ejemplo, gatos, perros, aviones, etc.) basándose en las características de la imagen de entrada.

    - Existen otras funciones de activación comunes, como la función sigmoid, la función tanh, y la función softmax, cada una de las cuales se utiliza para diferentes propósitos según el problema específico de la red neuronal.

- 'optimizer' se refiere a la función de optimización para ajustar los pesos conforme a la taza de aprendizaje que se utilizará en la capa densa que se está agregando a la red neuronal.

In [16]:
import tensorflow as tf
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=200, activation="relu"))
ann.add(tf.keras.layers.Dense(units=200, activation="relu")) 
ann.add(tf.keras.layers.Dense(units=200, activation="relu"))
ann.add(tf.keras.layers.Dense(units=2, activation="softmax"))
ann

In [17]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

In [18]:
# se usa sparse_categorical_crossentropy
ann.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"], )

In [19]:
import time
import memory_profiler
start_time = time.time() 
%load_ext memory_profiler
%memit ann.fit(x=X_train, y=y_train, batch_size=32, epochs=50, validation_split=0.2, callbacks=[early_stopping])
end_time = time.time()
print(f"Time taken to fit the model: {end_time - start_time} seconds")

Epoch 1/50
70/70 [==============================] - 14s 178ms/step - loss: 0.1534 - accuracy: 0.9789 - val_loss: 0.0069 - val_accuracy: 0.9982
Epoch 2/50
70/70 [==============================] - 11s 163ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0179 - val_accuracy: 0.9928
Epoch 3/50
70/70 [==============================] - 11s 162ms/step - loss: 7.0387e-04 - accuracy: 0.9996 - val_loss: 0.0109 - val_accuracy: 0.9946
Epoch 4/50
70/70 [==============================] - 12s 166ms/step - loss: 0.0063 - accuracy: 0.9987 - val_loss: 0.0057 - val_accuracy: 0.9982
peak memory: 7194.84 MiB, increment: 580.71 MiB
Time taken to fit the model: 51.474650382995605 seconds


In [20]:
test_loss, test_accuracy = ann.evaluate(X_test, y_test)
print('Test accuracy:', test_accuracy)

38/38 [==============================] - 0s 12ms/step - loss: 0.0019 - accuracy: 0.9992
Test accuracy: 0.999162495136261


Si quisieramos guardar los pesos del modelo para no reentrenar el modelo sería de esta manera

NameError: name 'k' is not defined